<a id='intro'></a>

# Tools for exploring ICOS Atmospheric data


This notebook includes functions in Python for exploring ICOS data. The notebook is divided in the following parts:

- [Import Python  modules](#import_modules)
- [Station info](#station_info)
- [Text formatting function](#text_format)
- [Plotting functions](#plot_func)
- [Widget functions](#widget_func)




Use the links to quickly navigate to the parts you are interested in. 

<a id='import_modules'></a>
<br>
<br>

## 1. Import modules

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}


In [ ]:
#Import modules:
import sys
import numpy as np
import pandas as pd
from ipywidgets import Dropdown, ColorPicker, VBox, HBox, Output, Button, Checkbox, Tab, Label
from IPython.display import clear_output
from bokeh.io import show, output_notebook, reset_output
from bokeh.models import Range1d
from bokeh.models.tickers import FixedTicker

#Set the notebook as the selected output location:
reset_output()
output_notebook()

#Set path to ICOS tools:
#sys.path.insert(0,'/home/karolina/testpy/pytools')

#Import ICOS tools:
from icoscp.station import station
from icoscp.sparql import sparqls
from icoscp.sparql.runsparql import RunSparql
from icoscp.cpb.dobj import Dobj



<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='station_info'></a>

<br>
<br>

## 2. Get station info for selected domain

In [ ]:
# Temporary fix to filter out non-icos atmospheric stations. This will be removed once the pylib is updated.
from icoscp.station import station
from icoscp.sparql.runsparql import RunSparql


query = \
'''
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
prefix cpmeta: <http://meta.icos-cp.eu/ontologies/cpmeta/>
prefix cpst: <http://meta.icos-cp.eu/ontologies/stationentry/>
select ?uri ?id ?name ?icosClass ?country ?siteType ?lat ?lon ?elevation ?stationTheme ?firstName ?lastName ?email
from <http://meta.icos-cp.eu/resources/icos/>
from <http://meta.icos-cp.eu/resources/extrastations/>
from <http://meta.icos-cp.eu/resources/stationentry/>
where {
    {
        select ?uri ?id (sample(?pers) as ?piOpt)  where{
            ?uri cpmeta:hasStationId ?id .
            OPTIONAL{
                ?memb cpmeta:atOrganization ?uri ; cpmeta:hasRole <http://meta.icos-cp.eu/resources/roles/PI> .
                filter not exists {?memb cpmeta:hasEndTime []}
                ?pers cpmeta:hasMembership ?memb
            }
        }
        group by ?uri ?id
    }
    ?uri a ?stationTheme .
    bind(coalesce(?piOpt, <http://dummy>) as ?pi)
    OPTIONAL{
        ?pi cpmeta:hasFirstName ?firstName .
        ?pi cpmeta:hasLastName ?lastName
    }
    OPTIONAL{?pi cpmeta:hasEmail ?email}
    OPTIONAL {
        {
            ?provSt cpst:hasProductionCounterpart ?prodUriStr .
            filter(iri(?prodUriStr) = ?uri)
        }
        ?provSt cpst:hasSiteType ?siteType .
    }
    OPTIONAL {?uri cpmeta:hasStationClass ?icosClass}
    OPTIONAL {?uri cpmeta:hasName ?name  }
    OPTIONAL {?uri cpmeta:countryCode ?country }
    OPTIONAL {?uri cpmeta:hasLatitude ?lat }
    OPTIONAL {?uri cpmeta:hasLongitude ?lon }
    OPTIONAL {?uri cpmeta:hasElevation ?elevation }
}
'''
# Fetch all stations using the query above.
all_stations = RunSparql(sparql_query=query, output_format='pandas').run().transpose()
icos_atm_stations = list()
for index in all_stations:
    station_info = all_stations[index]
    # Only stations with the correct theme and icos class will be included.
    if station_info.stationTheme == 'http://meta.icos-cp.eu/ontologies/cpmeta/AS' and station_info.icosClass is not None:
        icos_atm_stations.append(station.get(station_info.id))
atm_stations = icos_atm_stations
# End of temporary fix.

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='station_info'></a>

<br>
<br>

## 3. Text formatting function

In [ ]:
def printmd(string):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 10 12:00:00 2019
    Last Changed:     Fri May 10 12:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that allows you to print the string input parameter using
                      markdown formatting code.
                      
    Input parameters: String of characters
                      (var_name: 'string', var_type: String)

    Output:           String     
    
    """
    
    #import module:
    from IPython.display import Markdown, display
    
    #Enable displaying string with markdown code:
    display(Markdown(string))

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='plot_func'></a>

<br>
<br>

## 4. Plotting functions
This part contains the Python code for two plotting functions: 

- interactive bokeh plot for ICOS AS
- interactive folium map with markers (marker info for ICOS stations)

In [ ]:
def add_bokeh_label(fig, x_coor, y_coor, unit, txt_str, txt_font_size, alignment):
    
    """
    Project:       ICOS Carbon Portal
    Date created:  2020-08-19
    Last modified: 2020-08-19
    Developed by:  Karolina
    
    Definition:    Function that adds a label to a Bokeh plot at a defined location.
    
    Input:         1. Bokeh plot object (var_name: "fig", var_type: python object)
                   2. x coordinate of the bottom left part of the label (var_name: "x_coor", var_type: float)
                   3. y coordinate of the bottom left part of the label (var_name: "y_coor", var_type: float)
                   4. unit of x- & y-coordinates e.g. "screen" (var_name: "unit", var_type: string)
                   5. label content - text (var_name: "txt_str", var_type: string)
                   6. font size of text in label (var_name: "txt_font_size", var_type: float)
                   7. alignment of label based on x- & y-coord, e.g. "below" 
                     (var_name: "alignment", var_type: string)
    
    Output:       The function adds a label to an existent plot object but does not return any values.
    """
    
    #Import modules:
    from bokeh.models import Label
    
    #Set the label position:
    label_opts = dict(x=x_coor, y=y_coor, x_units=unit, y_units=unit)
    
    #Create the label object:
    caption = Label(text=txt_str, **label_opts)
    
    #Format label object - set label-text font size:
    caption.text_font_size = txt_font_size
    
    #Add label to plot:
    fig.add_layout(caption, alignment) 
        

In [ ]:
# Below are some utility functions that parse data from dobj.meta.
def dobj_column_unit(dobj, column_label):
    # Returns: The unit of the data of the column, if applicable 
    df_var = dobj.variables
    return df_var.loc[df_var.name == column_label].unit.iloc[0] or ''
    

def dobj_column_type(dobj, column_label):
    # Returns: The a type for the column
    df_var = dobj.variables
    return df_var.loc[df_var.name == column_label].type.iloc[0] or ''

    
def dobj_sampling_height(dobj):
    # Returns: A string for the sampling height of the measured data, if applicable 
    
    try:
        if dobj.meta['specificInfo']['acquisition']['samplingHeight'] > 0:
            return '(' + str(dobj.meta['specificInfo']['acquisition']['samplingHeight']) + ' m.a.g.l.)' 
        else:
            return ''
    except Exception: 
        return ''


def dobj_station_name(dobj):
    # Returns: The station name where the data were sampled
    try:
        return str(dobj.station['org']['name'])
    except Exception:
        return ''
    
    
def dobj_product(dobj):
    # Returns: product if any
    try:
        return str(dobj.meta['specification']['self']['label'])
    except:
        return ''

In [ ]:
def plot_icos_as_qualinfo(dobj, station_dict, var, glyph, color='#0F0C08'):
    
    
    #Import modules:
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, BoxAnnotation, Title
    from bokeh.layouts import layout
    from ipywidgets import Textarea, Output, ToggleButton, VBox
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    
    #Check if dobj has data:
    if ((isinstance(dobj.get(), pd.DataFrame))&(var in dobj.get().columns.values)):
        
        #Get dataframe with data for current dobj:
        df = dobj.get()
        
        #Remove "-" from variable column name:
        new_var = var.replace('-','')
        
        #Replace var-column name in dataframe:
        df.rename(columns={var:new_var}, inplace=True)
        
        #Create a ColumnDataSource object:
        source = ColumnDataSource( data = df )
               
        # For the y-label we want the unit if any:
        unit = dobj_column_unit(dobj,var)
        if unit > '':
            unit = '(' + unit + ')'
        y_label = var.upper().translate(SUB) + ' ' + unit
            
        
        #Create a figure object:
        p = figure(plot_width=900,
                   plot_height=400,
                   x_axis_label='Time (UTC)', 
                   y_axis_label=y_label,
                   x_axis_type='datetime',
                   y_range=list(set(df[new_var])),
                   tools='pan,box_zoom,wheel_zoom,undo,redo,reset,save')
        
        # Set titles (some of the below might be the empty string):
        title_main = dobj_station_name(dobj)
        title_middle = dobj_product(dobj)
        
        column_type = dobj_column_type(dobj,var)
        sampl_height = dobj_sampling_height(dobj)
        if column_type == '' or sampl_height == '':
            title_last = column_type + sampl_height 
        else: 
            title_last = column_type + ' - ' + sampl_height 
        
        p.add_layout(Title(text=title_last, text_font_style="italic"), 'above')
        p.add_layout(Title(text=title_middle, text_font_size="13pt"), 'above')
        p.add_layout(Title(text=title_main, text_font_size="16pt"), 'above')


        
        #Check glyph:
        if(glyph=='line'):
            
            #Create circle glyph:
            g0 = p.circle('TIMESTAMP',new_var, source=source, radius=.02, color=color)
            
            #Create line glyph:
            g1 = p.line('TIMESTAMP',new_var, source=source, line_width=1, color=color)
            
        else:
            
            #Create circle glyph:
            g2 = p.circle('TIMESTAMP',new_var, source=source, size=4, alpha=0.5, color=color)
            

        #Set title attributes:
        p.title.align = 'center'
        p.title.text_font_size = '13pt'
        p.title.offset = 15

        #Set label font style:
        p.xaxis.axis_label_text_font_style = 'normal'
        p.yaxis.axis_label_text_font_style = 'normal'
        p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
        p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units
        
        #Set grid format:
        p.grid.grid_line_alpha = 0
        p.ygrid.band_fill_color = "olive"
        p.ygrid.band_fill_alpha = 0.1
        
        
        #Add label with copyright info:
        add_bokeh_label(p, 0, 10, 'screen', "© ICOS ERIC", '8pt', 'below')
        
        #Add label with license info:
        add_bokeh_label(p, 770, 23, 'screen', "CC4BY", '8pt', 'below')
        
        #Deactivate hover-tool, which is by default active:
        p.toolbar.active_inspect = None
        
        #Add hover tool:
        p.add_tools(HoverTool(tooltips=[("Time (UTC)","@TIMESTAMP{%Y-%m-%d %H:%M:%S}"),
                                        (var, "@"+new_var),],
                              formatters={"@TIMESTAMP": "datetime",},
                              # display a tooltip whenever the cursor is vertically in line with a glyph
                              mode="vline"))  
        
        #Set widget format
        text_layout = {'width':'100%', 'height':'140px', 'border':'2px solid lightblue'}
        btn_style2 = {'button_color':'#3973ac'}
        btn_layout2 = {'width':'45%', 'align_self':'center'}
        
        #Add Textarea-widget with quality-flag info:
        t = Textarea(value='U: data correct before manual quality control\nN: data incorrect before manual quality control\nO: data correct after manual quality control\nK: data incorrect after manual quality control\n\nDetailed info in [Hazan et. al, 2016], doi:10.5194/amt-9-4719-2016',
                     disabled=True,
                     layout=text_layout)
        
        #Add Togglebutton-widget to control the visibility of the Textarea-widget:
        tb = ToggleButton(value=False,
                          description='Quality flag description',
                          disabled=False,
                          button_style='success', 
                          tooltip='Click to view/hide description of quality flag values',
                          icon='check', # (FontAwesome names without the `fa-` prefix)
                          style = btn_style2,
                          layout= btn_layout2)
        
        #Create output-widget to store Textarea-widget:
        out = Output()
        
        #Function that controls the visibility of the Textarea-widget
        #based on the toogle-button status:
        def togglebttn_qualinfo(button_c):

            #Open output-object:
            with out:
                
                #Delete previous content:
                out.clear_output()
                
                #Check status of Tooglebutton:
                if(tb.value==True):
                    
                    #Display Textarea-widget:
                    display(t)

                else:
                    
                    #Clear output:
                    out.clear_output()


        #Add event-listener for Togglebutton:
        tb.observe(togglebttn_qualinfo, 'value')


        
        #return plot:
        return [p, VBox([out, tb])]
    
    else:
        
        #Failure to create plot:
        return False


In [ ]:
def plot_icos_single_station_binary(dobj, station_dict, var, glyph, color='#0F0C08'):
    
    
    #Import modules:
    import numpy as np
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, Title
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    
    #Check if dobj has data:
    if ((isinstance(dobj.get(), pd.DataFrame))&(var in dobj.get().columns.values)):
        
        #Get dataframe with data for current dobj:
        df = dobj.get()
        
        #Check if selected var is "st dev":
        if (any([sub_str in var for sub_str in ['Stdev']])):

            #Replace st dev-value "-9,999" to NaN:
            df[df[var] < 0] = np.nan
        
                
        #Check if column for selected variable includes only NaNs:
        if (df[var].isnull().all()==False):
        
            #Remove "-" from variable column name:
            new_var = var.replace('-','')

            #Replace var-column name in dataframe:
            df.rename(columns={var:new_var}, inplace=True)


            #Create a ColumnDataSource object:
            source = ColumnDataSource( data = df )

            # For the y-label we want the unit if any:
            unit = dobj_column_unit(dobj,var)
            if unit > '':
                unit = '(' + unit + ')'
            y_label = var.upper().translate(SUB) + ' ' + unit

            #Create a figure object:
            p = figure(plot_width=900,
                       plot_height=400,
                       x_axis_label='Time (UTC)', 
                       y_axis_label=y_label,
                       x_axis_type='datetime',
                       tools='pan,box_zoom,wheel_zoom,undo,redo,reset,save')
            # Set titles (some of the below might be the empty string):
            title_main = dobj_station_name(dobj)
            title_middle = dobj_product(dobj)

            column_type = dobj_column_type(dobj,var)
            sampl_height = dobj_sampling_height(dobj)
            if column_type == '' or sampl_height == '':
                title_last = column_type + sampl_height 
            else: 
                title_last = column_type + ' - ' + sampl_height 

            p.add_layout(Title(text=title_last, text_font_style="italic"), 'above')
            p.add_layout(Title(text=title_middle, text_font_size="13pt"), 'above')
            p.add_layout(Title(text=title_main, text_font_size="16pt"), 'above')

            

            #Check glyph:
            if(glyph=='line'):

                #Create circle glyph:
                g0 = p.circle('TIMESTAMP',new_var, source=source, radius=.02, color=color)

                #Create line glyph:
                g1 = p.line('TIMESTAMP',new_var, source=source, line_width=1, color=color)

            else:

                #Create circle glyph:
                g2 = p.circle('TIMESTAMP',new_var, source=source, size=4, alpha=0.5, color=color)


            #Set title attributes:
            p.title.align = 'center'
            p.title.text_font_size = '13pt'
            p.title.offset = 15

            #Set label font style:
            p.xaxis.axis_label_text_font_style = 'normal'
            p.yaxis.axis_label_text_font_style = 'normal'
            p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
            p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

            #Set grid format:
            p.grid.grid_line_alpha = 0
            p.ygrid.band_fill_color = "olive"
            p.ygrid.band_fill_alpha = 0.1

            #Add label with copyright info:
            add_bokeh_label(p, 0, 10, 'screen', "© ICOS ERIC", '8pt', 'below')

            #Add label with license info:
            add_bokeh_label(p, 770, 24, 'screen', "CC4BY", '8pt', 'below')


            #Deactivate hover-tool, which is by default active:
            p.toolbar.active_inspect = None
            
            

            #Check if the selected variable is time:
            if (any([sub_string in var for sub_string in ['time', 'TIME']])):
                
                #Add hover tool:
                p.add_tools(HoverTool(tooltips=[("Time (UTC)","@TIMESTAMP{%Y-%m-%d %H:%M:%S}"),
                                                ("Time (UTC)", "@"+new_var+"{%Y-%m-%d %H:%M:%S}"),],
                                      formatters={"@TIMESTAMP": "datetime",
                                                  "@"+new_var: "datetime",},
                                      # display a tooltip whenever the cursor is vertically in line with a glyph
                                      mode="vline"))    
                
                
            else:
                #Add hover tool:
                p.add_tools(HoverTool(tooltips=[("Time (UTC)","@TIMESTAMP{%Y-%m-%d %H:%M:%S}"),
                                                (var, "@"+new_var+"{0.00}"),],
                                      formatters={"@TIMESTAMP": "datetime",},
                                      # display a tooltip whenever the cursor is vertically in line with a glyph
                                      mode="vline"))    

            #return plot:
            return p
        
        else:
            return False
            
    else:
        return False


In [ ]:
def plotmap(stations, selected_station, basemap, d_icon='cloud', icon_col='orange'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Thu Jun 25 02:00:00 2020
    Last Changed:     Thu Jun 25 02:00:00 2020
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a list of station objects containing info about ICOS Stations
                      the 3-character long station code of a selected station, the basemap type, the
                      marker icon and the marker color as input and returns an interactive Folium Map, with
                      the location of the selected station highlighted in red. 
                      Folium (URL): https://python-visualization.github.io/folium/quickstart.html
                      
    Input parameters: 1. Dataframe with Information regarding ICOS Stations
                         (var_name: 'stations', var_type: List)
                      2. Station 3-character Code
                         (var_name: 'selected_station', var_type: String)
                      3. Type of basemap (e.g. OSM or imagery)
                         (var_name: 'basemap', var_type: String)
                      4. Marker icon name (domain specific)
                         (var_name: 'd_icon', var_type: String)
                      5. Marker color (domain specific)
                         (var_name: 'icon_col', var_type: String)

    Output:           Folium Map (Folium Map Object)
    
    """
    
    
    #Import modules:
    import folium
    import branca
    
    #Check what type of basemap is selected:
    if(basemap=='Imagery'):
        
        #Create folium map-object:
        m = folium.Map(location=[station.get(selected_station).lat, station.get(selected_station).lon],
               zoom_start=5,
               tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
               attr = 'Esri',
               name = 'Esri Satellite',
               overlay = False,
               control = True)
        
    else:
        #Create folium map-object:
        m = folium.Map(
            location=[station.get(selected_station).lat, station.get(selected_station).lon],
            zoom_start=4)

    #Add marker-tooltip:
    tooltip = 'Click to view station info'

    def add_marker(map_obj, st_dict, marker_txt, marker_color):

        #Add popup text:
        popup=folium.Popup(marker_txt,
                           parse_html=True,
                           max_width=400)

        #Create marker and add it to the map:
        folium.Marker(location=[float(st_dict['lat']),
                                float(st_dict['lon'])],
                      popup=popup,
                      icon=folium.Icon(color=marker_color, icon=d_icon),
                      tooltip=tooltip).add_to(map_obj)



    #Create markers for all stations except selected station:
    for st in stations:
        
        #Get station info in html-format and add it to an iframe:
        iframe = branca.element.IFrame(html=st.info('html'), width=350, height=400)
        
        
        #Check if current station is selected station:
        if(st.info()['stationId']==selected_station):
            
            #Add marker for selected station:
            add_marker(m, st.info(), iframe, 'darkred')
        
        else:
            #Add marker for current station to map:
            add_marker(m, st.info(), iframe, icon_col) 

    

    #Show map:
    display(m)



<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='widget_func'></a>

<br>
<br>

## 5. Widget-form functions

In [ ]:
#Create function that displays a widget-form:
def icos_as_widgtform():
    
    #Function that updates station-widget values
    #based on the selection of data product value:
    def init_station_widget(val):
    
        #Get sorted list of stations that have data for the selected data product:
        station_ls = sorted([(s.info()['name'], s)
                             for s in atm_stations
                             if isinstance(s.products(), pd.DataFrame)
                             if val['new'] in s.products().values],key=lambda x: x[0])

        #Udpate widget-values:
        stations_wdgt.options = station_ls
##########################################################################################
     
    
    def init_var_widget(data_product, station, sampl_height):
        
        # 2022-03-16: Did not work if combination of product, station and sampling height had no dataobject associated with it 
        try:
            #Get a dataframe containing the colName & valueType for every variable:
            var_df = Dobj(station.data().dobj.loc[(station.data().specLabel==data_product)&
                                                       (station.data().samplingheight==str(sampl_height))].iloc[0]).variables[['name','type']] 

            #Create a sorted list of tuples with the valueType and colName of every variable (we sort by name unless the variable is a ghg/radon radiation):
            list_of_ghg = ['co2','co','n2o','ch4','14c','rn']
            var_ls = sorted([(var_df.type.iloc[i] + ' (' + var_df.name.iloc[i] + ')', 
                              var_df.name.iloc[i])
                             for i in range(len(var_df))], 
                            key = lambda v: '0' if v[1] in list_of_ghg else v[1].upper())
            
            button_exe.disabled = False
        except:
            # this can happen if the dobj has unretrievable data 
            var_ls= []
            button_exe.disabled = True

        #Update widget-values:
        var_wdgt.options = var_ls  
##########################################################################################
    
    
    def on_change_station_widget(val):

        #Filter sampling-height-widget values based on selected data product:
        sampl_height_wdgt.options = val['new'].sh(prods_wdgt.value)
##########################################################################################


    def on_change_prods_widget(val):

        #Call fubction to filter the station-widget values:
        init_station_widget(val)

        #Call function to filter the sampling-height-widget values:
        on_change_station_widget({'new':stations_wdgt.value})

        #Update variable-widget values:
        
        if val['new'] is not None:
            new_value = val['new']
        else:
            new_value = ''
        init_var_widget(new_value, stations_wdgt.value, sampl_height_wdgt.value)
##########################################################################################

    #Set widget format options:
    wdgt_layout = {'width':'390px', 'min_width':'400px'}
    lbl_layout = {'width':'150px'}
    cit_lbl_layout = {'width':'50px'}
    btn_layout = {'width':'50%', 'height':'30px', 'align_self':'center', 'margin':'20px 0px 40px 0px'}
    btn_style = {'button_color':'#3973ac'}

        
        
    #Create dropdown list for data products
    ## In the list of products we will remove 
    ## the "false_products" below.
    ## - Products with different data. 
    ##  we remove these (the ones present today) 
    ##  in a "hard coded" way.
    ##  A better, but slower, choice would be to
    ##  validate each product p by checking the 
    ##  the variables of the dobj of each station
    ##  with the product p.  
    ## - Here we will not look into STILT-data, 
    ##   for this we recommend the user to look 
    ##   at the pylib_examples 6a-6c  
        
    
    false_products = ['Lagrangian Transport Model Result', 'Atmospheric measurements results archive', 'STILT mole fraction time series']
    products = pd.DataFrame()
    t = [products.append(s.products()) for s in atm_stations if isinstance(s.products(), pd.DataFrame)]
    prod = sorted(list( p for p in products.append(t)[0].unique() if not p in false_products))
    
    prods_wdgt = Dropdown(options = prod, value=prod[0], layout=wdgt_layout)
    
    #Add event-handler (products):
    prods_wdgt.observe(on_change_prods_widget, 'value')

    #Create station list:
    stations_ls = sorted([(atm_stations[i].info()['name'], atm_stations[i])
                          for i in range(len(atm_stations))],key=lambda x: x[0])

    #Create dropdown list for ICOS stations:
    stations_wdgt = Dropdown(options = stations_ls, layout=wdgt_layout)

    #Initialize station-widget values:
    init_station_widget({'new':prods_wdgt.value})

    #Add dropdown for station sampling height:
    sampl_height_wdgt = Dropdown(options = stations_wdgt.value.sh(prods_wdgt.value), layout=wdgt_layout)

    #Add event-handler (stations):
    stations_wdgt.observe(on_change_station_widget, 'value') 

    #Create button widget (execution):
    # 2022-03-16: moved up so that the button could be disabled in case of no variables available
    button_exe = Button(description='Update plot',
                        disabled=False,
                        button_style='danger', 
                        tooltip='Press the button to update plot',
                        icon='check',
                        layout=btn_layout,
                        style=btn_style)

    #Add variable-widget (displays selection of variables to be plotted):
    # 2022-03-16: Did not work if combination of product, station and sampling height had no dataobject associated with it 
    try:
        # Note that the 
        stn_df = stations_wdgt.value.data()
        
        var_df = Dobj(stn_df.dobj.loc[(stn_df.specLabel==prods_wdgt.value)&
                                      (stn_df.samplingheight==str(sampl_height_wdgt.value))].iloc[0]).variables[['name','type']] 
        
        #Create a sorted list of tuples with the valueType and colName of every variable (we sort by name unless the variable is a GHG):
        list_of_ghg = ['co2','co','n2o','ch4','14c','rn']
        var_ls = sorted([(var_df.type.iloc[i] + ' (' + var_df.name.iloc[i] + ')',
                          var_df.name.iloc[i]) 
                         for i in range(len(var_df))], key = lambda v: '0' if v[1] in list_of_ghg else v[1].upper())
        
        var_wdgt = Dropdown(options = var_ls, layout=wdgt_layout)

        #Initialize variable-widget values:
        init_var_widget(prods_wdgt.value, stations_wdgt.value, sampl_height_wdgt.value)
        
        button_exe.disabled = False

    except: 
        #should not be the case since we removed these products
        var_wdgt = Dropdown(options = [], layout=wdgt_layout)
        button_exe.disabled = True
        
    
    #Add colorpicker-widget:
    colorpicker = ColorPicker(concise=False, value='#3973ac', disabled=False, layout=wdgt_layout)

    #Add glyph-type widget:
    glyph_wdgt = Dropdown(options = ['circle', 'line'], layout=wdgt_layout)

    #Add checkbox-widget for citation:
    citation=Checkbox(value=True, disabled=False, layout=wdgt_layout)
    
    #Create tabs with widgets:
    tab = Tab(children=[VBox([HBox([Label('Data product: ', layout=lbl_layout), prods_wdgt]),
                              HBox([Label('Station: ', layout=lbl_layout), stations_wdgt]),
                              HBox([Label('Sampling height: ', layout=lbl_layout), sampl_height_wdgt]),
                              HBox([Label('Variable: ', layout=lbl_layout), var_wdgt])]),
                        VBox([HBox([Label('Glyph color: ', layout=lbl_layout), colorpicker]),
                              HBox([Label('Glyph type: ', layout=lbl_layout), glyph_wdgt])]),
                        HBox([Label('Citation: ', layout=cit_lbl_layout), citation])])


    #Set tab title:
    tab.set_title(0, 'Data')
    tab.set_title(1, 'Plot format')
    tab.set_title(2, 'Citation')

    #Function that executes on button_click (calculate score):
    def on_exe_bttn_clicked(button_c):
        
        #Get data object for selected station, sampling height and data product:
        dobj_df = stations_wdgt.value.data().dobj.loc[(stations_wdgt.value.data().specLabel==prods_wdgt.value)&
                                                      (stations_wdgt.value.data().samplingheight==str(sampl_height_wdgt.value))]
        
        #Check if selected variable refers to a quality control variable:
        if(var_wdgt.value in ['AP-Flag', 'AT-Flag', 'RH-Flag', 'WD-Flag', 'WS-Flag', 'Flag', 'UserFlag']):
            #Call quality flag plotting-function:
            
            p = plot_icos_as_qualinfo(Dobj(dobj_df.iloc[0]), stations_wdgt.value, var_wdgt.value, glyph_wdgt.value, colorpicker.value)
            
        #If selected variabls is not a quality control variable:
        else:
            #Call main plotting-function:
            p = plot_icos_single_station_binary(Dobj(dobj_df.iloc[0]), stations_wdgt.value, var_wdgt.value, glyph_wdgt.value, colorpicker.value)
            
        #Add plot to output object:
        with plot_out:
            
            #Delete previous output:
            clear_output()

            print('\n')

            if(p!=False):
                
                if(var_wdgt.value in ['AP-Flag', 'AT-Flag', 'RH-Flag', 'WD-Flag', 'WS-Flag', 'Flag', 'UserFlag']):
                    
                    #Add plot to output object:
                    show(p[0], notebook_handle=True)
                    
                    #Add widgets:
                    display(p[1])
                
                else:
                    #Add plot to output object:
                    show(p, notebook_handle=True)

                if(citation.value):

                    #Print citation title:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                    #Print APA-style citation:
                    printmd("<sub>"+Dobj(dobj_df.iloc[0]).citation+"</sub>")
                
                #export_png(p, filename="plot.png") 
            else:
                print("\033[0;31;1m No data or no valid data available for the selected data product.\033[0;31;0m\n\n")



    #Create form object:
    form_out = Output()

    #Create output object:
    plot_out = Output()

    #Call function on button_click-event (calculate score):
    button_exe.on_click(on_exe_bttn_clicked)


    #Open output obj:
    with form_out:

        #Clean previous values:
        clear_output()

        #Show plot:
        display(VBox([tab, button_exe, plot_out]))

    #Display form:
    display(form_out)

In [ ]:
#Create function for ICOS AS map widget form:
def map_wdgt_form_icos_AS():

    #Create dropdown list for ICOS stations:
    stations_wdgt = Dropdown(options = sorted([(atm_stations[i].info()['name'], atm_stations[i])
                                               for i in range(len(atm_stations))],key=lambda x: x[0]),
                             description='Station')
    
    #Create dropdown list basemap options:
    basemap_wdgt = Dropdown(options = ['Imagery', 'OpenStreetMap'], description='Basemap')
    
    #Create button widget (execution):
    button_exe = Button(description='Update map',
                        disabled=False,
                        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                        tooltip='Press the button to update map',
                        icon='check')
    
    
    #Format widgets:
    stations_wdgt.layout.width = '379px'
    stations_wdgt.style.description_width = 'initial'
    stations_wdgt.layout.margin = '2px 2px 2px 238px'
    basemap_wdgt.layout.width = '394px'
    basemap_wdgt.style.description_width = 'initial'
    basemap_wdgt.layout.margin = '2px 2px 2px 224px'
    button_exe.style.button_color = '#3973ac'
    button_exe.layout.width = '250px'
    button_exe.layout.margin = '50px 100px 40px 330px'
    
    #Create form object:
    form_out = Output()

    #Create output object:
    plot_out = Output()
    
    #Function that executes on button_click (calculate score):
    def on_exe_bttn_clicked(button_c):
    
        #Open output object:
        with plot_out:

            #Delete previous output:
            clear_output()

            #Show new map:
            plotmap(atm_stations, stations_wdgt.value.info()['stationId'], basemap_wdgt.value, d_icon='cloud', icon_col='orange')

    #Call function on button_click-event (calculate score):
    button_exe.on_click(on_exe_bttn_clicked)

    #Open output obj:a
    with form_out:

        #Clean previous values:
        clear_output()

        #Show plot:
        display(VBox([stations_wdgt, basemap_wdgt, button_exe, plot_out]))

    #Display form:
    display(form_out)

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>